# Open Ocean
# Open Earth Fundation

<h1> Step 2: calculate different metrics for each modulating factor </h1>

This notebook is the second part of the `Step1_Curate_IUCN_RedList.ipynb`

<h2> Modulating Factor 1: Normalize Biodiversity Score </h2>

Species diversity refers to the variety of different species present in a given area, as well as their abundance and distribution. This includes the number of species, their relative abundances, and how evenly or unevenly distributed they are.
Our proposal is: apply the Simpson and Shannon Index to obtain a local value of the MPA and normalize each sqd km value

### Data needed for this project

- Species names
- Species abundance
- Species distribution

Next Steps:

1. Find a database or datasets with abundance and distribution information for the entire ACMC
2. If it isn't reallistic, try to simulate that data

Options:
1. IUCN RED List and simulate abundance information
2. GBIF species information and simulate abundance and distribution information

### 1. Importing libraries.

In [3]:
# load basic libraries
import os
import glob
import boto3

import math
import numpy as np
import pandas as pd

# to plot
import matplotlib.pyplot as plt

# to manage shapefiles
import shapely
import geopandas as gpd
from shapely.geometry import Polygon, Point, box
from shapely.ops import linemerge, unary_union, polygonize

In [4]:
import fiona; #help(fiona.open)

**Import OEF functions**

In [5]:
%load_ext autoreload

In [6]:
#Run this to reload the python file
%autoreload 2
from MBU_utils import *

### 2. Load data

In [7]:
ACMC = gpd.read_file('https://ocean-program.s3.amazonaws.com/data/raw/MPAs/ACMC.geojson')

In [8]:
df = gpd.read_file('/Users/maureenfonseca/Desktop/Data-Oceans/ACMC_IUCN_data/gdf_ACMC_IUCN_range_status_filtered.shp')

In [9]:
grid = create_grid(ACMC, grid_shape="hexagon", grid_size_deg=1.)

### 3. Preliminary calculations


In [10]:
len(df)

627

**Shannon Index**

$\text{H} = -\sum[{p_i}\times\ln(p_i)]$

where, pi is the proportion of the entire community made up of species i

In [11]:
#Polygons of species distribution to be clipped to roi
df2 = gpd.clip(df.set_crs(epsg=4326, allow_override=True), ACMC)

In [12]:
df2 = df2[0:100]

In [23]:
#Join in a gdf all the geometries within ROI
joined = gpd.clip(df2.set_crs(epsg=4326, allow_override=True), ACMC)
    
#Count the number of overlappong geometries from joined gdf
overlap_geo = count_overlapping_geometries(joined)
    
#This is to count how many geometries are in each grid 
merged2 = gpd.sjoin(overlap_geo, grid, how='left')
merged2['n_species']= overlap_geo['count_intersections']

#Compute stats per grid cell
dissolve = merged2.dissolve(by="index_right", aggfunc="count")

#Put this into cell
grid.loc[dissolve.index, 'n_species'] = dissolve.n_species.values
    
#Calculate the normalize factor 
#normalized_factor = grid_gdf['n_habitats']/grid_gdf['n_habitats'].max()
    
#Convert area from degrees to square kilometers
#this case apply only for Central America
#https://epsg.io/31970
#grid_gdf['area_sqkm'] = (grid_gdf.to_crs(crs=31970).area)*10**(-6)
#grid_gdf['mbu_habitat_survey'] = normalized_factor*grid_gdf['area_sqkm']

/Users/maureenfonseca/Desktop/Oceans/oceanprogram/Marine_Ecosystem_Credits/Marine_Biodiversity/MBU_Methodology/MBU_utils.py:140: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if this_row_boundary.type[:len('multi')].lower() == 'multi':
/Users/maureenfonseca/Desktop/Oceans/oceanprogram/Marine_Ecosystem_Credits/Marine_Biodiversity/MBU_Methodology/MBU_utils.py:161: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  new_gdf['geom_centroid'] = new_gdf.centroid


In [ ]:
grid